# About

Here we collect our dataset for detecting AI-generated content. 

-------------------

A list of possible domains to fetch content from:

**Current Total:** (71.967 📝)

- Politics
    - [German Bundestag](https://www.bundestag.de/) ✔ (10394 📝)
    - [House of Commons](https://reshare.ukdataservice.ac.uk/854292/) ✔ (5944 📝)
- Student/School
    - [Kaggle Student Essays](https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2/data?select=train.csv) ✔ (9107 📝)
    - Take the english dataset and translate it?
- Research
    - [Arxiv](https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=language+model&terms-0-field=all&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2005&date-to_date=2020&date-date_type=submitted_date&abstracts=show&size=50&order=-announced_date_first) ✔ (5395 📝)
    - [I've seen a paper recently that creates full AI written papers. Maybe I can use it.](https://www.kaggle.com/discussions/general/527817#2958636)
- News
    - [Spiegel Online](https://www.spiegel.de/) ✔ (14543 📝)
    - [CNN Articles](https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail) ✔ (10601 📝)
- Blogs/Tutorials/Forums
- Law
    - [German Open Legal Data](https://de.openlegaldata.io/) ✔
    - [European Court of Human Rights Cases](https://www.kaggle.com/datasets/mathurinache/ecthrnaacl2021/data) ✔ (7276 📝)
- Philosophy
    - Gutenberg Project ([ENG](https://www.gutenberg.org/) | [GER](https://www.projekt-gutenberg.org/)) ✔
- Literature
    - Gutenberg Project ([ENG](https://www.gutenberg.org/) | [GER](https://www.projekt-gutenberg.org/)) ✔ (4103 📝)
- Blogs, Food and Lifestyle
    - [Food Blogs](https://detailed.com/food-blogs/)
    - [WebBlogs](https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus?select=blogtext.csv) (ENG) ✔ (9999 📝)
- Religion
    - [Bible](https://github.com/mxw/grmr/blob/master/src/finaltests/bible.txt) (ENG|GER) ✔
- Gaming

Interesting Languages:

- English
- German

In [ ]:
import sys
import importlib
import os
import pandas as pd
import time
import argparse

from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv("../.env")

True

In [2]:
class CONFIG:
    SRC_ROOT_PATH = os.getenv('SRC_ROOT_PATH')
    SRC_ROOT_PATH_COLL = os.getenv('SRC_ROOT_PATH_COLL')
    DATA_ROOT_PATH = os.getenv('DATA_ROOT_PATH')

    RUN_AS_SCRIPT = False

    COLLECT = False

    SYNCHRONIZE_TO_MONGODB = False
    CLEAN_SYNTHESIZED_ITEMS = False

    SYNTHESIZE = True
    FORCE_SYNTH = False
    SYNTHESIZE_PER_COLLECTOR = 100000

    EXTRACT_FEATURES = False
    FORCE_FEATURED = False
    EXTRACT_FEATURES_PER_COLLECTOR = 15
    BATCH_SIZE = 10

    SKIP_PER_COLLECTOR = 0

    CAUSAL_LLMS = []

In [ ]:
# # So that it includes local imports. This is some next level python shit import
# sys.path.insert(0, CONFIG.SRC_ROOT_PATH)
# sys.path.insert(0, CONFIG.SRC_ROOT_PATH_COLL)

In [3]:
import data_collector
import data_collector.collector
import data_collector.collected_item
import data_collector.collectors.bundestag_collector
import data_collector.collectors.house_of_commons_collector
import data_collector.collectors.student_essays_collector
import data_collector.collectors.arxiv_collector
import data_collector.collectors.spiegel_collector
import data_collector.collectors.cnn_news_collector
import data_collector.collectors.open_legal_data_collector
import data_collector.collectors.euro_court_cases_collector
import data_collector.collectors.religion_collector
import data_collector.collectors.gutenberg_collector
import data_collector.collectors.blog_corpus_collector

import data_collector.agents.ai_agent
import data_collector.agents.ollama_agent
import data_collector.agents.openai_agent
import data_collector.agents.ollama_agent

import data_collector.mongodb_client

def reload():
    importlib.reload(data_collector)
    importlib.reload(data_collector.collector)
    importlib.reload(data_collector.collected_item)
    importlib.reload(data_collector.collectors.bundestag_collector)
    importlib.reload(data_collector.collectors.house_of_commons_collector)
    importlib.reload(data_collector.collectors.student_essays_collector)
    importlib.reload(data_collector.collectors.arxiv_collector)
    importlib.reload(data_collector.collectors.spiegel_collector)
    importlib.reload(data_collector.collectors.cnn_news_collector)
    importlib.reload(data_collector.collectors.open_legal_data_collector)
    importlib.reload(data_collector.collectors.euro_court_cases_collector)
    importlib.reload(data_collector.collectors.religion_collector)
    importlib.reload(data_collector.collectors.gutenberg_collector)
    importlib.reload(data_collector.collectors.blog_corpus_collector)

    importlib.reload(data_collector.agents.ai_agent)
    importlib.reload(data_collector.agents.openai_agent)
    importlib.reload(data_collector.agents.ollama_agent)

    importlib.reload(data_collector.mongodb_client)

reload()

ModuleNotFoundError: No module named 'collected_item'

We basically always need access to our db.

In [5]:
mongo_conn = data_collector.mongodb_client.MongoDBConnection(os.getenv('MONGO_DB_CONNECTION'))

Connecting to the MongoDB...
Connected to MongoDB successfully.


Check if we have cli arguments passed in. This is only done when this notebook is turned to a script.

In [ ]:
parser = argparse.ArgumentParser(description="Worker Script for the orchestrator.")

parser.add_argument("--collectors", nargs="+", help="Pass in the collectors of this instance. Only works if --script-mode=True")
parser.add_argument("--collect", action="store_true", help="Determine if we collect a dataset.")
parser.add_argument("--synth", action="store_true", help="Determine if we synthesize a dataset.")
parser.add_argument("--synch", action="store_true", help="Determine if we syncrhonize the dataset to the mongo db.")
parser.add_argument("--clean", action="store_true", help="Determine if we clean the collected synthesized items.")

parser.add_argument("--take", type=int, default=100, help="Determine how many items we synthesize per collector.")
parser.add_argument("--skip", type=int, default=0, help="Determine how many items we skip per collector.")
parser.add_argument("--batch_size", type=int, default=0, help="Determine how many items we skip per collector.")

parser.add_argument("--featured", action="store_true", help="Determine if we feature_exract a dataset.")
parser.add_argument("--causal_llms", help="Pass in the CausalLLMs to generate the features.")

parser.add_argument("--force", action="store_true", help="Determine if we force a synthesization.")

try:
    args = parser.parse_args()
    print('=== Passed Parameters:')
    print('= Collectors:')
    print(args.collectors)

    print(f'= Take: {str(args.take)}')
    CONFIG.EXTRACT_FEATURES_PER_COLLECTOR = args.take
    CONFIG.SYNTHESIZE_PER_COLLECTOR = args.take
    
    print(f'= Skip: {str(args.skip)}')
    CONFIG.SKIP_PER_COLLECTOR = args.skip

    print(f'= Batch Size: {str(args.batch_size)}')
    CONFIG.BATCH_SIZE = args.batch_size
    
    print(f'= Force: {str(args.force)}')
    CONFIG.FORCE_SYNTH = args.force
    CONFIG.FORCE_FEATURED = args.force

    print(f'= Synchronize to DB: {str(args.synch)}')
    CONFIG.SYNCHRONIZE_TO_MONGODB = args.synch
    print(f'= Clean synthesized items: {str(args.synch)}')
    CONFIG.CLEAN_SYNTHESIZED_ITEMS = args.clean

    print(f'= Activated Levels:')
    print(f'- Collect: {args.collect}')
    CONFIG.COLLECT = args.collect
    print(f'- Synthesize: {args.synth}')
    CONFIG.SYNTHESIZE = args.synth
    print(f'- Extract features: {args.featured}')
    print(f'- Used CausalLLMs: {args.causal_llms}')
    CONFIG.CAUSAL_LLMS = args.causal_llms.split(',')
    CONFIG.EXTRACT_FEATURES = args.featured
    
    CONFIG.RUN_AS_SCRIPT = True
except:
    print('CLI parsing failed - this is hence run as a notebook.')


---------------------

# Collect

First, we gotta create a dataset and for that, we collect texts from various domains.

In [7]:
# If this is run as a script, that means the orcestrator passes in a list of collectors we are supposed to use.
# Fill the list dynamically then. If its from within a notebook, just add them by hand.
if CONFIG.RUN_AS_SCRIPT:
    collection = []
    for collector_name in args.collectors:
        try:
            module_name, class_name = collector_name.rsplit(".", 1)
            CollectorClass = getattr(eval(module_name), class_name)
            collector_instance = CollectorClass(CONFIG.DATA_ROOT_PATH)
            collection.append(collector_instance)
        except AttributeError as e:
            print(f"Error creating collector '{collector_name}': {e}")
else:
    collection = [
        collectors.bundestag_collector.BundestagCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.house_of_commons_collector.HouseOfCommonsCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.student_essays_collector.StudentEssaysCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.arxiv_collector.ArxivCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.spiegel_collector.SpiegelCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.cnn_news_collector.CNNNewsCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.open_legal_data_collector.OpenLegalDataCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.euro_court_cases_collector.EuroCourtCasesCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.religion_collector.ReligionCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.gutenberg_collector.GutenbergCollector(CONFIG.DATA_ROOT_PATH),
        #collectors.blog_corpus_collector.BlogCorpusCollector(CONFIG.DATA_ROOT_PATH)
    ]

if CONFIG.RUN_AS_SCRIPT:
    print(f'Dynamically created collectors: {[type(c).__name__ for c in collection]}')
else:
    print(f'Manually defined collectors: {[type(c).__name__ for c in collection]}')

Manually defined collectors: ['BundestagCollector']


In [8]:
if CONFIG.COLLECT:
    total_items = 0

    for coll in collection:
        try:
            coll.init()
            coll.collect(CONFIG.FORCE_FEATURED)
            total_items += coll.get_count()
        except Exception as ex:
            print('ERROR: Current collection failed due to an error: ')
            print(ex)
            print('\n ***** Continuing with the other collectors. ***** \n')

    print('\n\n ==================================================== \n\n')
    print(f'All collectors finished. Total data items: {total_items}')
else:
    print('Collecting turned off - skipping it.')

We started collecting items to disc and then switched to a MongoDB. Hence this script.

In [9]:
if CONFIG.SYNCHRONIZE_TO_MONGODB:
    # Use this when synchronizing only the collected level.
    if False:
        for coll in collection:
            print(f'Doing collector: {coll.get_folder_path()} with {coll.get_count()} items.')

            for df in coll.read_from_cache('collected'):
                try:
                    items = [collected_item.CollectedItem.from_dict(row) for _, row in df.iterrows() if row['text'] != '' and row['text'] is not None]
                    mongo_conn.insert_many_collected_items(items)
                except Exception as ex:
                    print('Error while inserting a CollectedItem: ', ex)
    
    # Use this when synchornizing already synthesized items, mainly the ones by gpt-4-o
    if True:
        for coll in collection:
            print(f'Doing collector: {coll.get_folder_path()} with {coll.get_count()} items.')

            for df in coll.read_from_cache('synthesized'):
                for index, row in df.iterrows():
                    try:
                        item = collected_item.CollectedItem.from_dict(row)
                        if item.synthetization is None or len(item.synthetization) == 0 or item.text == '':
                            continue

                        found = False
                        for mongo_item in mongo_conn.get_collected_items_by_text(item.text):
                            found = True
                            coll_item = collected_item.CollectedItem.from_dict(mongo_item)
                        
                            if any(synth['agent'] == 'gpt-4o-mini' for synth in coll_item.synthetization):
                                continue                                
                        
                            coll_item.synthetization.append(item.synthetization[0])
                            mongo_conn.update_collected_item(coll_item)

                        if found is False:
                            print(f'Item with id {item.id} had no stored equivalent in the db.')
                    except Exception as ex:
                        print('Error while inserting a CollectedItem: ', ex)
                print('Done with a dataframe chunk.')


---------------------

# Synthesize

After we've collected the dataset, we want to create it's AI-generated counterpart. We do this on multiple levels:

- Inject passages of AI-generated content
- Trying to rewrite the whole text as an AI agent.
- Trying different models

In [10]:
agents = [
    # data_collector.agents.openai_agent.OpenAIAgent(name='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY')),
    # data_collector.agents.openai_agent.OpenAIAgent(name='o3-mini', api_key=os.getenv('OPENAI_API_KEY')),
    # data_collector.agents.openai_agent.OpenAIAgent(name='gpt-4-turbo', api_key=os.getenv('OPENAI_API_KEY')),
    #data_collector.agents.ollama_agent.OllamaAgent(name='nemotron', 
    #                                               ollama_base_url=os.getenv('OLLAMA_BASE_URL'),
    #                                               context_length=131072),
    data_collector.agents.ollama_agent.OllamaAgent(name='phi3:3.8b', 
                                                   ollama_base_url=os.getenv('OLLAMA_BASE_URL'),
                                                   context_length=131072),
    #data_collector.agents.ollama_agent.OllamaAgent(name='gemma2:9b', 
    #                                               ollama_base_url=os.getenv('OLLAMA_BASE_URL'),
    #                                               context_length=8192),
    #data_collector.agents.ollama_agent.OllamaAgent(name='deepseek-r1:32b', 
    #                                               ollama_base_url=os.getenv('OLLAMA_BASE_URL'),
    #                                               context_length=131072),
]

Foreach agent, we go through all different collectors and synthesize the texts.

In [11]:
if CONFIG.SYNTHESIZE:
    # Foreach collector (so foreach domain)
    for coll in tqdm(collection, desc='Collectors', leave=False):
        items_count = 0
        for mongo_item in mongo_conn.get_collected_items_by_domain(coll.get_folder_path(), skip=CONFIG.SKIP_PER_COLLECTOR, batch_size=10):
            if items_count >= CONFIG.SYNTHESIZE_PER_COLLECTOR:
                break

            item = collected_item.CollectedItem.from_dict(mongo_item)
            
            if item.text == '' or item.text is None:
                continue
            
            # We have multiple agents. Foreach agent, synthesize the item
            for agent in agents:
                # Skip if an agent if that one already did the synthesization
                if any(synth['agent'] == agent.name for synth in item.synthetization):
                    continue            

                try:
                    new_obj = {
                        'agent': agent.name,
                        'synth_obj': agent.synthesize_collected_item(item, coll)
                    }
                    item.synthetization.append(new_obj)
                except Exception as ex:
                    print(f'There was an error with item with id {item.id} from collected {coll.get_folder_path()}')
                    print(ex)
            items_count += 1
            # Afterwards, update and save the changes to the db.
            mongo_conn.update_collected_item(item)

    print('Done with synthesization.')
else:
    print('Synthesization turned off - skipping it.')

Collectors:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

---------------------

# Feature Extraction

After we have collected and synthesized a dataset, we can now start collecting the feature space. We do that with the Detector class, give him an ensemble of models and let it extract features.

In [10]:
import detector
import causal_lm

importlib.reload(detector)
importlib.reload(causal_lm)

<module 'causal_lm' from '/mnt/e/Python_Projects/PrismAI/PrismAI/src/causal_lm.py'>

Create the detector, which we use for feature extractio and also for prediction if we wanted (currently not). Also test the detection, which is also the feature extraction.

I want to try out the following models maybe:
- [openGPT-X/Teuken-7B-instruct-research-v0.4](https://huggingface.co/meta-llama/Llama-3.2-3B)
- GPT2 as a cheap alternative maybe
- [meta-llama/Llama-3.2-3B](https://huggingface.co/meta-llama/Llama-3.2-3B)

In [11]:
if CONFIG.EXTRACT_FEATURES:
    if CONFIG.RUN_AS_SCRIPT:
        det = detector.Detector(models=CONFIG.CAUSAL_LLMS, min_token_length=16)
    else:
        det = detector.Detector(models=['GPT2'], min_token_length=16)

    test_text = "In his first State of the Union address, President Joe Biden delivered an optimistic outlook on the nation's economy, celebrating its remarkable rebound while drawing a clear contrast with the policies of his predecessor, Donald Trump. Against the backdrop of ongoing challenges posed by the pandemic and geopolitical tensions, Biden sought to instill confidence in the American people and reaffirm his administration's commitment to fostering inclusive growth and resilience."
    print(det.detect(test_text))

Created model GPT2 to device cuda:0
Model ensemble: ['GPT2']

{'metadata': {'full_text': "In his first State of the Union address, President Joe Biden delivered an optimistic outlook on the nation's economy, celebrating its remarkable rebound while drawing a clear contrast with the policies of his predecessor, Donald Trump. Against the backdrop of ongoing challenges posed by the pandemic and geopolitical tensions, Biden sought to instill confidence in the American people and reaffirm his administration's commitment to fostering inclusive growth and resilience.", 'ensemble': ['GPT2'], 'sample_rate': 6, 'sample_sequence_length': 12, 'min_token_length': 16, 'date': '2025-01-06 11:28:43.184215', 'seed': 42, 'signature': '06389dcec0049767d1dea3176ad6a88b3ef63782aa9628f537f3396c8272d1d7'}, 'ensemble_results': [{'model': 'GPT2', 'sample_results': [{'avg_prob': 0.40746395699776866, 'model_outputs': {'generated_text': ' Biden sought to instill confidence in the American people and reaff', 'step

If everything works, we can go through the collectors and extract the features from each text.

In [12]:
def extract_features(text):
    try:
        fs = det.detect(text, sample_rate=16, sample_sequence_length=32, k=100)
        fs['metadata']['full_text'] = ''
        return fs
    except Exception as ex:
        print(ex)
        return None
    
def write_featured_items(featured_items, stored_path):
    out_df = pd.DataFrame([item.__dict__ for item in featured_items])
    out_df.to_json(stored_path)
    print(f'Stored and wrote {len(out_df)} featured items to file {stored_path}')

In [13]:
if CONFIG.EXTRACT_FEATURES:
    for coll in tqdm(collection, desc='Collectors', leave=False):
        
        synth_items_df = pd.concat(coll.get_synthesized_items_dfs(), ignore_index=True)
        items_count = 0
        featured_items = []
        meta = coll.get_meta_file()
        if 'total_feature_extracted' not in meta:
            meta['total_feature_extracted'] = 0
        possible_skip = meta['total_feature_extracted']

        if CONFIG.FORCE_FEATURED:
            possible_skip = 0

        print(f'Loaded in {len(synth_items_df)} synthesized items.')
        print(f'Already featured {possible_skip} items.')
        print(f'Since force is {CONFIG.FORCE_FEATURED}, items count starts at: {str(possible_skip)}')

        for index, row in synth_items_df.iterrows():
            
            if items_count < possible_skip:
                items_count += 1
                continue

            if items_count >= CONFIG.EXTRACT_FEATURES_PER_COLLECTOR:
                print(f'Featured {items_count} items; going to next collector.')
                break

            item = collected_item.CollectedItem.from_dict(row)

            # Extract features from this item here.
            try:
                # Do the original human text first.
                item.feature_space = extract_features(item.text)

                # Then the synth objects
                if item.synthetization is not None:
                    # We have multiple AI agents that did a synthesization
                    for agent_obj in item.synthetization:
                        synth_obj = agent_obj['synth_obj']
                        if synth_obj is None:
                            continue
                        # We did synthesize chunks and fulltexts.
                        chunks = synth_obj['synth_chunks']
                        fulltext = synth_obj['synth_fulltext']
                        if chunks is not None:
                            chunks['feature_space'] = extract_features(chunks['synth_text'])
                        if fulltext is not None:
                            fulltext['feature_space'] = extract_features(fulltext['synth_text'])
                featured_items.append(item)
            except Exception as ex:
                print('Error while extracting features from an item:')
                print(ex)

            items_count += 1

            # We write and store in batches.
            if len(featured_items) >= CONFIG.BATCH_SIZE:
                stored_path = os.path.join(coll.get_feature_space_path(), f"items_{items_count}.json")
                write_featured_items(featured_items, stored_path)
                featured_items = []
                # Update the meta file.
                meta['total_feature_extracted'] = items_count
                coll.update_meta_file(meta)

        # and the final batch
        if len(featured_items) > 0:
            stored_path = os.path.join(coll.get_feature_space_path(), f"items_{items_count}.json")
            write_featured_items(featured_items, stored_path)
            # Update the meta file.
            meta['total_feature_extracted'] = items_count
            coll.update_meta_file(meta)


Collectors:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3392 > 1024). Running this sequence through the model will result in indexing errors


Loaded in 1816 synthesized items.
Already featured 0 items.
Since force is False, items count starts at: 0
The text must be at least 48 tokens long, got only 0.
Stored and wrote 10 featured items to file /mnt/c/home/projects/prismAI/data/bundestag/feature_space/items_10.json
The text must be at least 48 tokens long, got only 44.
Featured 15 items; going to next collector.
Stored and wrote 5 featured items to file /mnt/c/home/projects/prismAI/data/bundestag/feature_space/items_15.json
